For this lab I didnt add my own collected tracks from the last lab (lab-api-wrappers) because it ran for ages and didnt finish in time. Also, i didnt know which Kaggle dataset to pick? However, i assume the pipeline would be pretty much the same as to what i did now.

PS. I would really like some feedback on how i could make it a bit prettier (in particular the code in the case that the user_track is not in the dataset)

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="609b2df1fce5457eaf050c6c16248fa0",
                                                           client_secret="624f5460987444a28e9c449f5e4d77ac"))

In [3]:
url = "https://www.billboard.com/charts/hot-100"
r = requests.get(url)

In [4]:
r.status_code 

200

In [5]:
soup = BeautifulSoup(r.content, "html.parser")

In [6]:
# song titles
songs = soup.find_all("span", class_="chart-element__information__song text--truncate color--primary")

# artists
artists = soup.find_all("span", class_="chart-element__information__artist text--truncate color--secondary")

# last week
last_week_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--last")

# peak rank
peak_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--peak")

# weeks on chart
weeks_on_chart = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--week")

In [7]:
for i in [songs, artists, last_week_ranks, peak_ranks, weeks_on_chart]:
    for j in range(len(i)):
        i[j] = i[j].getText()

In [8]:
billboard = pd.DataFrame(
    {"song": songs,
     "artist": artists,
     "last_week_rank": last_week_ranks,
     "peak_rank": peak_ranks,
     "weeks_on_chart": weeks_on_chart}
)

In [9]:
billboard

,song,artist,last_week_rank,peak_rank,weeks_on_chart
0,Stay,The Kid LAROI & Justin Bieber,2,1,13
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11
2,Fancy Like,Walker Hayes,5,3,16
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5
4,Bad Habits,Ed Sheeran,6,2,15
...,...,...,...,...,...
95,Esta Danada,Ivan Cornejo,-,96,1
96,Whiskey And Rain,Michael Ray,-,97,1
97,TSU,Drake,84,9,5
98,Love All,Drake Featuring JAY-Z,87,10,5


In [10]:
all_tracks = []
all_tracks_ids = []
for track in billboard['song']:
    results = sp.search(q='track: {0}'.format(track), limit=1)
    all_tracks.append(results)
    for track in results:
        track_id = results['tracks']['items'][0]['id']
        all_tracks_ids.append(track_id)

In [11]:
all_tracks

[{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A+Stay&type=track&offset=0&limit=1',
   'items': [{'album': {'album_type': 'single',
      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2tIP7SsRs7vjIcLrU85W8J'},
        'href': 'https://api.spotify.com/v1/artists/2tIP7SsRs7vjIcLrU85W8J',
        'id': '2tIP7SsRs7vjIcLrU85W8J',
        'name': 'The Kid LAROI',
        'type': 'artist',
        'uri': 'spotify:artist:2tIP7SsRs7vjIcLrU85W8J'},
       {'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'},
        'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s',
        'id': '1uNFoZAHBGtllmzznpCI3s',
        'name': 'Justin Bieber',
        'type': 'artist',
        'uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s'}],
      'available_markets': ['AD',
       'AE',
       'AG',
       'AL',
       'AM',
       'AO',
       'AR',
       'AT',
       'AU',
       'AZ',
       'BA',
       'BB',

In [12]:
all_tracks_ids

['5HCyWlXZPP0y6Gqq8TgA20',
 '27NovPIUIRrOZoCHxABJwK',
 '3ZozFqJJlDBNIm4xqxn2ZD',
 '0k1WUmIRnG3xU6fvvDVfRG',
 '6PQ88X9TkUIAUIZJHW2upE',
 '4ZtFanR9U6ndgddUvNcjcG',
 '748mdHapucXQri7IAO8yFK',
 '463CkQjx2Zk1yXoBuierM9',
 '2BcMwX1MPV6ZHP4tUT9uq6',
 '5FG7Tl93LdH117jEKYl3Cm',
 '75MNhvTCCKsST3YqqUiU9r',
 '3FeVmId7tL5YN8B7R3imoM',
 '3Vi5XqYrmQgOYBajMWSvCi',
 '37BZB0z9T8Xu7U3e65qxFy',
 '3USxtqRwSYz57Ewm6wWRMp',
 '3Wrjm47oTz2sjIgck11l5e',
 '1SC5rEoYDGUK4NfG82494W',
 '0k4d5YPDr1r7FX77VdqWez',
 '5f3MXmTmstozFg0BH1yPUk',
 '37Nqx7iavZpotJSDXZWbJ3',
 '5CZ40GBx1sQ9agT82CLQCT',
 '6OGogr19zPTM4BALXuMQpF',
 '3J4VKkUpLvomXJbn9ZAypO',
 '1ZIKpMTRPqhwAoVXWY3tXI',
 '5fwSHlTEWpluwOM0Sxnh5k',
 '0e8nrvls4Qqv5Rfa2UhqmO',
 '0nqhKXDjsyBMvbeWmgijD0',
 '3dV1fjKOlLnaNK5MeZHXGv',
 '4RVwu0g32PAqgUiJoXsdF8',
 '40iJIUlhi6renaREYGeIDS',
 '6HU7h9RYOaPRFeh0R3UeAr',
 '6zSpb8dQRaw0M1dK8PBwQz',
 '7BQE1P0NUalHSvFtrxasx9',
 '1sx0XsWUusUoPstbvokZjP',
 '2Xr1dTzJee307rmrkt8c0g',
 '4s7TIubkdsgdtDEDFEAYVL',
 '7MAibcTli4IisCtbHKrGMh',
 

In [13]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    track = [name, album, artist, release_date, length, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

In [14]:
def get_all_tracks_info(ids):
    all_tracks_info = []
    for i in range(len(ids)):
        time.sleep(.5)
        all_tracks_info.append(getTrackFeatures(ids[i]))
    return all_tracks_info

In [15]:
all_tracks_info = get_all_tracks_info(all_tracks_ids)

In [16]:
all_tracks_info

[['STAY (with Justin Bieber)',
  'STAY (with Justin Bieber)',
  'The Kid LAROI',
  '2021-07-09',
  141805,
  100,
  0.0383,
  0.591,
  0.764,
  0,
  0.103,
  -5.484,
  0.0483,
  169.928,
  4],
 ['INDUSTRY BABY (feat. Jack Harlow)',
  'INDUSTRY BABY (feat. Jack Harlow)',
  'Lil Nas X',
  '2021-07-23',
  212000,
  98,
  0.0203,
  0.736,
  0.704,
  0,
  0.0501,
  -7.409,
  0.0615,
  149.995,
  4],
 ['Fancy Like',
  'Country Stuff',
  'Walker Hayes',
  '2021-06-04',
  161853,
  84,
  0.114,
  0.649,
  0.76,
  0,
  0.317,
  -6.487,
  0.0574,
  80.018,
  4],
 ['Way 2 Sexy (with Future & Young Thug)',
  'Certified Lover Boy',
  'Drake',
  '2021-09-03',
  257604,
  93,
  0.000619,
  0.803,
  0.597,
  4.5e-06,
  0.323,
  -6.035,
  0.141,
  136.008,
  4],
 ['Bad Habits',
  'Bad Habits',
  'Ed Sheeran',
  '2021-06-25',
  231041,
  97,
  0.0469,
  0.808,
  0.897,
  3.14e-05,
  0.364,
  -3.712,
  0.0348,
  126.026,
  4],
 ['good 4 u',
  'SOUR',
  'Olivia Rodrigo',
  '2021-05-21',
  178146,
  96,
  

In [17]:
df = pd.DataFrame(all_tracks_info, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])

In [18]:
df.head()

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,STAY (with Justin Bieber),STAY (with Justin Bieber),The Kid LAROI,2021-07-09,141805,100,0.038300,0.591,0.764,0.000000,0.1030,-5.484,0.0483,169.928,4
1,INDUSTRY BABY (feat. Jack Harlow),INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,2021-07-23,212000,98,0.020300,0.736,0.704,0.000000,0.0501,-7.409,0.0615,149.995,4
2,Fancy Like,Country Stuff,Walker Hayes,2021-06-04,161853,84,0.114000,0.649,0.760,0.000000,0.3170,-6.487,0.0574,80.018,4
3,Way 2 Sexy (with Future & Young Thug),Certified Lover Boy,Drake,2021-09-03,257604,93,0.000619,0.803,0.597,0.000005,0.3230,-6.035,0.1410,136.008,4
4,Bad Habits,Bad Habits,Ed Sheeran,2021-06-25,231041,97,0.046900,0.808,0.897,0.000031,0.3640,-3.712,0.0348,126.026,4


In [19]:
billboard_with_features = pd.concat([billboard, df], axis=1)

In [20]:
billboard_with_features

,song,artist,last_week_rank,peak_rank,weeks_on_chart,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,Stay,The Kid LAROI & Justin Bieber,2,1,13,STAY (with Justin Bieber),STAY (with Justin Bieber),The Kid LAROI,2021-07-09,141805,100,0.038300,0.591,0.764,0.000000,0.1030,-5.484,0.0483,169.928,4
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11,INDUSTRY BABY (feat. Jack Harlow),INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,2021-07-23,212000,98,0.020300,0.736,0.704,0.000000,0.0501,-7.409,0.0615,149.995,4
2,Fancy Like,Walker Hayes,5,3,16,Fancy Like,Country Stuff,Walker Hayes,2021-06-04,161853,84,0.114000,0.649,0.760,0.000000,0.3170,-6.487,0.0574,80.018,4
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5,Way 2 Sexy (with Future & Young Thug),Certified Lover Boy,Drake,2021-09-03,257604,93,0.000619,0.803,0.597,0.000005,0.3230,-6.035,0.1410,136.008,4
4,Bad Habits,Ed Sheeran,6,2,15,Bad Habits,Bad Habits,Ed Sheeran,2021-06-25,231041,97,0.046900,0.808,0.897,0.000031,0.3640,-3.712,0.0348,126.026,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Esta Danada,Ivan Cornejo,-,96,1,Está Dañada,Alma Vacía,Ivan Cornejo,2021-08-20,214148,80,0.380000,0.405,0.565,0.019500,0.1070,-9.266,0.0372,159.892,3
96,Whiskey And Rain,Michael Ray,-,97,1,Whiskey And Rain,Whiskey And Rain,Michael Ray,2020-09-25,206506,72,0.508000,0.653,0.623,0.000003,0.2850,-4.560,0.0244,100.021,4
97,TSU,Drake,84,9,5,TSU,Certified Lover Boy,Drake,2021-09-03,308755,84,0.228000,0.568,0.560,0.000003,0.3900,-9.059,0.4200,130.019,4
98,Love All,Drake Featuring JAY-Z,87,10,5,Love All (with JAY-Z),Certified Lover Boy,Drake,2021-09-03,228461,84,0.354000,0.730,0.576,0.000036,0.1500,-5.442,0.2870,92.131,4


In [21]:
billboard_with_features.isna().sum()

song                0
artist              0
last_week_rank      0
peak_rank           0
weeks_on_chart      0
name                0
album               0
artist              0
release_date        0
length              0
popularity          0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
time_signature      0
dtype: int64

In [22]:
to_transform = billboard_with_features[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo']]

In [23]:
transformer = StandardScaler()
transformer_fit = transformer.fit(to_transform)
transformed = transformer_fit.transform(to_transform)

In [24]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(transformed)

KMeans(n_clusters=3, random_state=42)

In [25]:
clusters = kmeans.predict(transformed)
clusters

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0,
       0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2,
       0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2,
       0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0,
       2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2], dtype=int32)

In [26]:
pd.Series(clusters).value_counts().sort_index()

0    65
1     1
2    34
dtype: int64

In [27]:
billboard_with_features["cluster"] = clusters
billboard_with_features.head()

,song,artist,last_week_rank,peak_rank,weeks_on_chart,name,album,artist,release_date,length,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,cluster
0,Stay,The Kid LAROI & Justin Bieber,2,1,13,STAY (with Justin Bieber),STAY (with Justin Bieber),The Kid LAROI,2021-07-09,141805,...,0.038300,0.591,0.764,0.000000,0.1030,-5.484,0.0483,169.928,4,0
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11,INDUSTRY BABY (feat. Jack Harlow),INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,2021-07-23,212000,...,0.020300,0.736,0.704,0.000000,0.0501,-7.409,0.0615,149.995,4,0
2,Fancy Like,Walker Hayes,5,3,16,Fancy Like,Country Stuff,Walker Hayes,2021-06-04,161853,...,0.114000,0.649,0.760,0.000000,0.3170,-6.487,0.0574,80.018,4,0
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5,Way 2 Sexy (with Future & Young Thug),Certified Lover Boy,Drake,2021-09-03,257604,...,0.000619,0.803,0.597,0.000005,0.3230,-6.035,0.1410,136.008,4,0
4,Bad Habits,Ed Sheeran,6,2,15,Bad Habits,Bad Habits,Ed Sheeran,2021-06-25,231041,...,0.046900,0.808,0.897,0.000031,0.3640,-3.712,0.0348,126.026,4,0


In [28]:
kmeans.inertia_

513.7818084340386

In [29]:
kmeans = KMeans(n_clusters=3, random_state=1234, verbose=1, n_init=1)
kmeans.fit(transformed)
kmeans.inertia_

Initialization complete
Iteration 0, inertia 644.6160700857615
Iteration 1, inertia 554.4152647481951
Iteration 2, inertia 535.6393589939041
Iteration 3, inertia 529.2672739463669
Iteration 4, inertia 528.6463348838196
Iteration 5, inertia 528.2816088632334
Iteration 6, inertia 526.5495856164025
Iteration 7, inertia 525.8600829531039
Iteration 8, inertia 525.3491885175819
Iteration 9, inertia 522.8573956763221
Iteration 10, inertia 516.4654946462557
Iteration 11, inertia 516.2369737305771
Iteration 12, inertia 516.050126075001
Iteration 13, inertia 515.9086150521264
Converged at iteration 13: strict convergence.


515.9086150521264

In [30]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(16,8))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('k')
# plt.ylabel('inertia')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.show()

In [31]:
user_input = str(input("Enter tunes: "))

Enter tunes: Stay


In [32]:
def get_suggestion(user_input):
    for i in range(len(billboard_with_features)-1):
        if billboard_with_features['song'].iloc[i] == user_input:
            cluster = billboard_with_features['cluster'].iloc[i]
            for x in billboard_with_features['cluster']:
                if cluster == billboard_with_features['cluster'].iloc[x]:
                    suggestion = billboard_with_features['song']
                    billboard_with_features['suggestion'] = suggestion
            return cluster, billboard_with_features['suggestion'].sample(1).values[0]

In [33]:
user_output = get_suggestion(user_input)

In [34]:
user_output

(0, 'Way 2 Sexy')

In [35]:
user_input = str(input("Enter tunes: "))

Enter tunes: I Will Survive


In [36]:
def getTrackFeatures_user(id):
    features = sp.audio_features(id)
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    track = [acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo]
    return track

In [37]:
def get_all_tracks_info_user(ids):
    all_tracks_info = []
    for i in range(len(ids)):
        time.sleep(.5)
        all_tracks_info.append(getTrackFeatures_user(ids[i]))
    return all_tracks_info

In [38]:
def get_suggestion(user_input):
    for i in range(len(billboard_with_features)-1):
        if billboard_with_features['song'].iloc[i] == user_input:
            cluster = billboard_with_features['cluster'].iloc[i]
            for x in billboard_with_features['cluster']:
                if cluster == billboard_with_features['cluster'].iloc[x]:
                    suggestion = billboard_with_features['song']
                    billboard_with_features['suggestion'] = suggestion
            return cluster, billboard_with_features['suggestion'].sample(1).values[0]
        elif user_input not in range(len(billboard_with_features['song'])-1):
            user_track = []
            user_track_id = []
            user_track_all_info = []
            results = sp.search(q='track: {0}'.format(user_input), limit=1)
            user_track.append(results)
            for track in results:
                track_id = results['tracks']['items'][0]['id']
                user_track_id.append(track_id)
                user_track_all_info = get_all_tracks_info_user(user_track_id)
                to_transform = user_track_all_info
                user_transformed = transformer_fit.transform(to_transform)
                cluster_user_track = kmeans.predict(user_transformed)[0]
                for x in billboard_with_features['cluster']:
                    if cluster_user_track == billboard_with_features['cluster'].iloc[x]:
                        suggestion = billboard_with_features['song']
                        billboard_with_features['suggestion'] = suggestion
                return cluster_user_track, billboard_with_features['suggestion'].sample(1).values[0]
                            

In [39]:
user_output = get_suggestion(user_input)

In [40]:
user_output

(2, 'Peaches')

In [41]:
user_input = str(input("Enter tunes: "))

Enter tunes: Stay


In [42]:
user_output = get_suggestion(user_input)

In [43]:
user_output

(0, 'Yonaguni')